# How-to train a model on Azure ML

This notebook takes you through the steps of training a model on Azure ML for The Ocean Cleanup. We train the models through Azure ML to provide us with a good registration of all performed tests, so that we can see why and how a model was created.

When the result of a training run is satisfactory, a model can be registered from there, from which point we can deploy it.

There are a few concepts to know about first:

- Workspace: The entire AzureML environment you are working in. The Workspace contains all the other elements.
- Experiment: A collection of Runs (see below). A logical container for training a model with different parameters to determine the best.
- Run: A single train/test run of a model. These are tied to an experiment. If you want to train the same model with different parameters, so you can compare them, these are different runs under the same experiment.
- Environment: The code environment used by your code. This contains things like the required Python packages. Multiple options exist here - from just using your local environment to completely curated environments directly from Azure.
- Dataset: A single dataset as registered in the AzureML workspace.

With that out of the way, lets dive right in. Looking at these components, our first step will be to get the correct Workspace:

In [1]:
from azurewrapper.workspace import get_workspace

subscription_id = "29d66431-a7ce-4709-93f7-3bdb01a243b3"
resource_group = "ExperimentationJayke"
workspace_name = "ExperimentationJayke"

workspace = get_workspace(subscription_id, resource_group, workspace_name)

## Create experiment

Now that we have a workspace available, we need to create an experiment. As describe above, an experiment will be the container for multiple runs, in which we can train and compare the model using different parameters.

The experiment needs a name. Use something that is descriptive and clear to anyone seeing this.

In [2]:
from azurewrapper.train import create_experiment
experiment = create_experiment(workspace, "model-yolo-v-1-0")

## Create or select compute target

We want to train our model on a GPU cluster on AzureML. Lets create one (or load an existing one).

In [3]:
from azurewrapper.compute import get_compute

compute_target = get_compute(workspace, "gpu-cluster", vm_size='STANDARD_NC6', max_nodes=4)

Found existing compute target


## Create the environment

We will now need to create an environment. In this case, we build the enviroment based of the Azure docker Tensorflow image, but with our own pip requirements.

In [4]:
from azurewrapper.environment import get_environment

environment = get_environment(
    workspace,
    "tensorflow-yolo",
    pip_requirements="./examples/yolo/Tensorflow_YOLO/requirements.txt",
    docker_image="mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04"
)

No environment with that name found, creating new one


## Prepare model wraper

Now it's time to perform our first Run of the experiment. However, before we can do this, we will need a wrapper around our model. This wrapper needs to do a few things:

- Initialize and train the model with:
  - The desired parameters
  - The desired data
- Evaluate the performance of the trained model
- Register the parameters and the performance in the Run object
- Add the generated model artifacts to the Run object

There is skeleton code for this available: `skeleton_files/train.py`. In this file you fill in what parameters you expect, you create and train and evaluate the model using these parameters and the loaded in dataset(s), and you register the results and the created artifacts with the Run.

For this how-to, we will use the example provided in `examples/yolo/train.py`. This is an implementation of the file mentioned above. It accepts 4 parameters: `LR_INIT`, `LR_END`, `WARMUP_EPOCHS` and `EPOCHS`, and requires `yolov4 weights` as dataset.

### Changes to tensorflow YOLO code

I made some changes to the Tensorflow YOLO code as provided by Mats/from GitHub:

- Changed the filepaths in `Tensorflow_YOLO/yolov3/configs.py`, in addition to other changes there as indicated
  by Mats
- I made the imports from subdirectories in `Tensorflow_YOLO/train.py`, `Tensorflow_YOLO/evaluate_mAP.py`, and all   files `Tensorflow_YOLO/yolov3` relative instead of absolute, so we can import from our training wrapper.
- `Tensorflow_YOLO/evaluate_mAP.py` also returns the FPS
- `Tensorflow_YOLO/train.py:main()` returns the evaluated mAP and FPS, and accepts the provided parameters as override to the config file.
- Added AzureML modules to `Tensorflow_YOLO/requirements.txt`

## Run the experiment

Now we need to create and run the experiment. First, we fetch the desired datasets, and combine these into train- and test sets. Note that we can provide multiple sets for both training and testing. Also note that each set consists of both a label and an image dataset.

In [5]:
from azureml.core import Dataset

train_images = Dataset.get_by_name(workspace, name="campaign-26-10-2020_images")
train_labels = Dataset.get_by_name(workspace, name="campaign-26-10-2020_labels")
test_images = Dataset.get_by_name(workspace, name="campaign-22-10-2020_images")
test_labels = Dataset.get_by_name(workspace, name="campaign-22-10-2020_labels")
trainsets = [
    (train_labels, train_images),
    (test_labels, test_images)
]
testsets = [
    (test_labels, test_images)
]

We now have everything we need to perform the run. Lets do so!

In [6]:
from azurewrapper.train import perform_run
from azureml.core.runconfig import TensorflowConfiguration

weights = Dataset.get_by_name(workspace, name="YOLO")

run = perform_run(experiment, 'train.py', 'examples/yolo', environment=environment,
                  trainsets=trainsets, testsets=testsets, compute_target=compute_target,
                  parameters={
                      'weights': weights.as_named_input('weights').as_mount(),
                      'LR_INIT': 1e-4,
                      'LR_END': 1e-6,
                      'WARMUP_EPOCHS': 2,
                      'EPOCHS': 100
                  })
run.wait_for_completion(show_output=True)

Submitting /home/jayke/dev/xomnia/TOC/AIDataPipeLine/ModelTraining/examples/yolo directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


RunId: model-yolo-v-1-0_1604919925_84762eb8
Web View: https://ml.azure.com/experiments/model-yolo-v-1-0/runs/model-yolo-v-1-0_1604919925_84762eb8?wsid=/subscriptions/29d66431-a7ce-4709-93f7-3bdb01a243b3/resourcegroups/ExperimentationJayke/workspaces/ExperimentationJayke

Streaming azureml-logs/55_azureml-execution-tvmps_dffe4d8dbdc114601f638f5fb28a8df77611bc5cded65c7a1d05ce27412b6bce_d.txt

2020-11-09T11:08:45Z Starting output-watcher...
2020-11-09T11:08:45Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2020-11-09T11:08:46Z Executing 'Copy ACR Details file' on 10.0.0.5
2020-11-09T11:08:46Z Copy ACR Details file succeeded on 10.0.0.5. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_ea71d6d6fbafdaeb54907902a21c77dc
171857c49d0f: Pulling fs layer
419640447d26: Pulling fs layer
61e52f862619: Pulling fs layer
c118dad7e37a: Pulling fs layer
2e36372995f9: Pulling fs layer
0b8e00a4ba4e: Pulling fs layer
c118dad7e37a: Waiting

epoch:11 step:    0/2, lr:0.000097, giou_loss:   2.45, conf_loss: 213.81, prob_loss:   3.51, total_loss: 219.77
epoch:11 step:    1/2, lr:0.000097, giou_loss:   2.55, conf_loss: 206.10, prob_loss:   3.51, total_loss: 212.16


giou_val_loss:   3.05, conf_val_loss: 182.64, prob_val_loss:   3.63, total_val_loss: 189.31


epoch:12 step:    0/2, lr:0.000097, giou_loss:   2.14, conf_loss: 199.54, prob_loss:   3.31, total_loss: 205.00
epoch:12 step:    1/2, lr:0.000097, giou_loss:   2.38, conf_loss: 195.78, prob_loss:   3.69, total_loss: 201.84


giou_val_loss:   3.84, conf_val_loss: 170.66, prob_val_loss:   4.94, total_val_loss: 179.43


epoch:13 step:    0/2, lr:0.000096, giou_loss:   2.47, conf_loss: 191.85, prob_loss:   3.40, total_loss: 197.72
epoch:13 step:    1/2, lr:0.000096, giou_loss:   3.27, conf_loss: 191.71, prob_loss:   4.03, total_loss: 199.01


giou_val_loss:   3.91, conf_val_loss: 161.12, prob_val_loss:   4.92, total_val_loss: 169.96


epoch:14 step:    0/2, lr:0.000096, giou

{'runId': 'model-yolo-v-1-0_1604919925_84762eb8',
 'target': 'gpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-11-09T11:08:38.893788Z',
 'endTimeUtc': '2020-11-09T11:15:21.837798Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '830e4226-38af-41c5-acf5-3066475e5154',
  'azureml.git.repository_uri': 'git@github.com:TheOceanCleanup/AIDataPipeLine.git',
  'mlflow.source.git.repoURL': 'git@github.com:TheOceanCleanup/AIDataPipeLine.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '1609c196a49515964b4c1022801125134f17dbf6',
  'mlflow.source.git.commit': '1609c196a49515964b4c1022801125134f17dbf6',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '0046c288-9c27-407a-afb5-d0439c374cbe'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'train_labels_0', 'me

In [8]:
run.register_model(
    "yolo_v4",
    model_path="outputs/checkpoint/",
    description="Yolo V4 implementation on Tensorflow",
    properties={
        "location": "somewhere",
        "time": "night"
    }
)

Model(workspace=Workspace.create(name='ExperimentationJayke', subscription_id='29d66431-a7ce-4709-93f7-3bdb01a243b3', resource_group='ExperimentationJayke'), name=yolo_v4, id=yolo_v4:1, version=1, tags={}, properties={'location': 'somewhere', 'time': 'night'})